In [1]:
%load_ext autoreload
%autoreload 2
from models.discriminator import CLIPSVMDiscriminator
from adversarial_models.CLIP_pgd import CLIPPGDAttack
from adversarial_models.CLIP_patch_pgd import CLIPPatchPGDAttack
import torch
import torchvision
import os

/Users/prachipatil/Desktop/CMU Courses/IDL/Project/CMU-IDL/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
source_dir = "subset/sources"
processed_dir = "subset/processed"
pgd_dir = "subset/pgd"
pgd_patch_dir = "subset/pgd_patch"
pgd_attn_dir = "subset/pgd_attn"

os.makedirs(processed_dir, exist_ok=True)
os.makedirs(pgd_dir, exist_ok=True)
os.makedirs(pgd_patch_dir, exist_ok=True)
os.makedirs(pgd_attn_dir, exist_ok=True)

model_path = "subset/CLIP_discriminator.pt"

In [5]:
model: CLIPSVMDiscriminator = torch.load(
    model_path, weights_only=False, map_location=torch.device("cpu")
)
model.device = "cpu"

In [6]:
discriminator = model.model
processor = model.processor

In [6]:
src_images = [
    torchvision.io.read_image(os.path.join(source_dir, filepath))
    for filepath in os.listdir(source_dir)
]
len(src_images)

6

In [15]:
# preprocesses and save images to processed_dir
paths = os.listdir(source_dir)
images = processor(images=src_images, return_tensors="pt")["pixel_values"]


for i, image in enumerate(images):
    torchvision.utils.save_image(image, os.path.join(processed_dir, paths[i]))

In [98]:
attack = CLIPPGDAttack(discriminator, model.svm, eps=8/255, random_start=True)

inpaths = [
    "subset/poster/srcs/dalle_148.jpg", "subset/poster/srcs/OpenJourney_9751.jpg"
]
outpaths =  [
    "subset/poster/e_8/dalle_148.jpg", "subset/poster/e_8/OpenJourney_9751.jpg"
]

images = [
    torchvision.io.read_image(filepath)
    for filepath in inpaths]
resize_transform = torchvision.transforms.Resize((224, 224))
images = torch.stack(images).to(discriminator.device)
images = resize_transform(images)
images = images.float() / 255.0  # Normalize to [0, 1]
images = resize_transform(images)
images = torch.FloatTensor(images)
# images = processor(images=images, return_tensors="pt")["pixel_values"]
target_label = torch.tensor([1]*len(outpaths)).to(discriminator.device)

# # Generate adversarial example
adversarial_image = attack(images, target_label)

for i in range(adversarial_image.shape[0]):
    # Get the image and filepath for the current index
    image = adversarial_image[i]
    filepath = outpaths[i]

    # Save the image using torchvision.utils.save_image
    torchvision.utils.save_image(image, filepath)

evaluate_dir("subset/poster/e_8")


Loss: 2.242676
Loss: -13.095894
Loss: -23.325668
Loss: -27.238659
Loss: -31.671082
Loss: -33.963562
Loss: -42.636475
Loss: -43.300907
Loss: -41.078064
Loss: -45.779037
Evaluating subset/poster/e_8
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
AUC: nan
mAP: 1.0000
Probabilities: [1. 1.]
Predictions: [1 1] (1 = real, 0 = fake)
Raw scores: [2.20022039 4.14677584]
Files: ['OpenJourney_9751.jpg', 'dalle_148.jpg']



/Users/prachipatil/Desktop/CMU Courses/IDL/Project/CMU-IDL/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/prachipatil/Desktop/CMU Courses/IDL/Project/CMU-IDL/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/prachipatil/Desktop/CMU Courses/IDL/Project/CMU-IDL/.venv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [ ]:
from adversarial_models.CLIP_pgd import preprocess_image
processed_custom = "subset/processed_custom"
os.makedirs(processed_custom, exist_ok=True)

transform = torchvision.transforms.Resize((224, 224))
# preprocess all images in source_dir and save to processed_custom 
for filepath in os.listdir(source_dir):
    image = preprocess_image(os.path.join(source_dir, filepath), transform)
    torchvision.utils.save_image(image, os.path.join(processed_dir, filepath))

In [103]:
from adversarial_models.CLIP_pgd import preprocess_image

# PGD Attack with patch
pgd_patch_dir = "subset/poster/patch"
os.makedirs(pgd_patch_dir, exist_ok=True)
os.makedirs(os.path.join(pgd_patch_dir, "boxes"), exist_ok=True)


patch_attack = CLIPPatchPGDAttack(discriminator, model.svm, patch_size=32, steps=20, num_patches=3, random_start=True)

inpaths = [
    "subset/poster/srcs/dalle_148.jpg", "subset/poster/srcs/OpenJourney_9751.jpg"
]
patch_paths =  [
    "subset/poster/patch/dalle_148.jpg", "subset/poster/patch/OpenJourney_9751.jpg"
]

box_paths =  [
    "subset/poster/patch/boxes_dalle_148.jpg", "subset/poster/patch/boxes_OpenJourney_9751.jpg"
]

images = [
    torchvision.io.read_image(filepath)
    for filepath in inpaths]
resize_transform = torchvision.transforms.Resize((224, 224))
images = torch.stack(images).to(discriminator.device)
images = resize_transform(images)
images = images.float() / 255.0  # Normalize to [0, 1]
images = resize_transform(images)
images = torch.FloatTensor(images)

# # Generate adversarial example
adversarial_image, boxes = patch_attack(images, target_label)

for i in range(adversarial_image.shape[0]):
    # Get the image and filepath for the current index
    image = adversarial_image[i]
    filepath = patch_paths[i]

    # Save the image using torchvision.utils.save_image
    torchvision.utils.save_image(image, filepath)
    torchvision.utils.save_image(boxes[i], box_paths[i])



Loss: 5.339697
Loss: 2.660016
Loss: 0.863431
Loss: 0.114115
Loss: -0.916858
Loss: -2.070945
Loss: -2.232011
Loss: -0.827594
Loss: -3.275060
Loss: -3.728092
Loss: -4.641870
Loss: -2.827746
Loss: -2.181078
Loss: -5.026191
Loss: -4.897472
Loss: -5.888506
Loss: -3.707053
Loss: -3.516158
Loss: -5.323720
Loss: -6.202042
Image 0: Selected 3 patches out of 3 requested
Image 1: Selected 3 patches out of 3 requested


In [104]:
evaluate_dir("subset/poster/patch")


Evaluating subset/poster/patch
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
AUC: nan
mAP: 1.0000
Probabilities: [3.29749077e-05 2.03776594e-01 1.51309128e-04 2.58396320e-01]
Predictions: [0 0 0 0] (1 = real, 0 = fake)
Raw scores: [-1.98646241 -0.30693305 -1.7007729  -0.24907872]
Files: ['boxes_OpenJourney_9751.jpg', 'boxes_dalle_148.jpg', 'OpenJourney_9751.jpg', 'dalle_148.jpg', 'boxes']



/Users/prachipatil/Desktop/CMU Courses/IDL/Project/CMU-IDL/.venv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [84]:
def evaluate_dir(dirname):
    print(f"Evaluating {dirname}")
    images = [
        torchvision.io.read_image(os.path.join(dirname, filepath))
        for filepath in os.listdir(dirname)
        if filepath.endswith(".png") or filepath.endswith(".jpg")
    ]
    labels = [0] * len(images)
    images = torch.stack(images)
    embeddings = model.run_clip(images)
    metrics = model.evaluate(embeddings, labels)
    raw_scores = model.svm.decision_function(embeddings)
    print("Probabilities:", metrics["probs"])
    print("Predictions:", metrics["preds"], "(1 = real, 0 = fake)")
    print("Raw scores:", raw_scores)
    print("Files:", os.listdir(dirname))
    print()
    

In [89]:
# evaluate_dir(source_dir)
evaluate_dir(pgd_dir)
evaluate_dir(pgd_patch_dir)

Evaluating subset/pgd
Accuracy: 0.4000
Precision: 1.0000
Recall: 0.4000
F1 Score: 0.5714
AUC: nan
mAP: 1.0000
Probabilities: [0.08024108 0.37689641 0.87471683 0.99999819 0.99999994]
Predictions: [0 0 1 1 1] (1 = real, 0 = fake)
Raw scores: [-0.50873735 -0.14664816  0.3127578   1.29143301  1.60828624]
Files: ['openjourney_v4_37056.jpg', 'stable_diff_9751.jpg', 'stable_diff_82434.jpeg', 'titan_83700.jpg', 'OpenJourney_9751.jpg', 'dalle_148.jpg']

Evaluating subset/pgd_patch_3


/Users/prachipatil/Desktop/CMU Courses/IDL/Project/CMU-IDL/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/prachipatil/Desktop/CMU Courses/IDL/Project/CMU-IDL/.venv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


RuntimeError: stack expects a non-empty TensorList

In [177]:
atack = CLIPPGDAttack(discriminator, model.svm, steps=3, eps=4/255, random_start=True)


In [8]:
# Add this after the attack but before using the results
import random
# walk through every image in images directory, run the PGD attack on it, and save the result to pgd_dir
# attack = CLIPPatchPGDAttack(discriminator, model.svm, eps=0.3, steps=20, patch_size=32, num_patches=5, random_start=True)
attack = CLIPPGDAttack(discriminator, model.svm, steps=3, eps=4/255, random_start=True)
preds = []
labels = []
resize_transform = torchvision.transforms.Resize((224, 224))
for dirpath in ["images/data/dalle", "images/data/real", "images/data/titan"]:
    filenames = os.listdir(dirpath)
    random.shuffle(filenames)
    for filename in filenames[:20]:
        try:
            filepath = os.path.join(dirpath, filename)
            if filepath.endswith(".png") or filepath.endswith(".jpg"):
                # Process the image
                print(filepath)
                if "real" not in filepath:
                    print("Adversarial example")
                    image = torchvision.io.read_image(filepath)
                    image = processor(images=image, return_tensors="pt")["pixel_values"]
                    target_label = torch.tensor([1]).to(discriminator.device)

                    # Generate adversarial example
                    adversarial_image = attack(image, target_label)
                    
             

                    embedding = model.run_clip(adversarial_image).reshape(1, -1)

                    pred, prob = model.predict_from_embeddings(embedding)

                    preds.append(pred.item())
                    labels.append(0)
                else:
                    print("Real example")
                    preds.append(1)
                    labels.append(1)
                
        except Exception as e:
            print(f"Error processing file {filename}: {e}")
            continue

images/data/dalle/dalle_73262.jpg
Adversarial example
Loss: 6.467362
Loss: -3.662686


It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Loss: -8.579271
images/data/dalle/dalle_37748.jpg
Adversarial example
Loss: 6.296868
Loss: 1.954516
Loss: -1.314785
images/data/dalle/dalle_20219.jpg
Adversarial example
Loss: 7.210421
Loss: 1.403381
Loss: -2.303245
images/data/dalle/dalle_37887.jpg
Adversarial example
Loss: 4.587760
Loss: 0.490902
Loss: -2.080090
images/data/dalle/dalle_48727.jpg
Adversarial example
Loss: 3.934434
Loss: 0.392909
Loss: -2.239207
images/data/dalle/dalle_84165.jpg
Adversarial example
Loss: 2.428377
Loss: -4.887422
Loss: -8.634383
images/data/dalle/dalle_19084.jpg
Adversarial example
Loss: 4.428191
Loss: 1.979040
Loss: 0.275106
images/data/dalle/dalle_58818.jpg
Adversarial example
Loss: 6.389173
Loss: 0.989652
Loss: 0.439508
images/data/dalle/dalle_50071.jpg
Adversarial example
Loss: 7.999412
Loss: -0.929499
Loss: -5.756628
images/data/dalle/dalle_42853.jpg
Adversarial example
Loss: 4.752370
Loss: -3.638960
Loss: -8.515268
images/data/dalle/dalle_90584.jpg
Adversarial example
Loss: 4.303026
Loss: -2.17674

In [9]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [10]:
preds

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [11]:

# calculate accuracy, precision, recall, and AUC
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
accuracy = accuracy_score(labels, preds)
precision = precision_score(labels, preds)
recall = recall_score(labels, preds)
auc = roc_auc_score(labels, preds)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("AUC:", auc)

Accuracy: 0.3333333333333333
Precision: 0.3333333333333333
Recall: 1.0
AUC: 0.5
